# Linear Regression (Multiple Variables) - Matrix Form

## What's Different from Single Variable?

In the previous notebook, we predicted house prices using just **one feature** (size). But real-world problems usually have **multiple features**!

For example, predicting house prices using:
- Size (sq ft)
- Number of bedrooms
- Age of house
- Distance to city center

### The Equation

**Single variable:** $y = mx + b$

**Multiple variables:** $y = w_1x_1 + w_2x_2 + w_3x_3 + w_4x_4 + b$

Where:
- $w_1, w_2, w_3, w_4$ are **weights** (like the slope $m$, but one for each feature)
- $x_1, x_2, x_3, x_4$ are our **features** (size, bedrooms, age, distance)
- $b$ is the **bias** (like the intercept)

## Matrix Notation (Don't Panic!)

Instead of writing $w_1x_1 + w_2x_2 + ...$, we use **matrices** to make it cleaner:

$$\boxed{y = Xw + b}$$

Where:
- $X$ is a matrix of all features (rows = samples, columns = features)
- $w$ is a vector of all weights
- $b$ is a scalar (single number)

### Why matrices?
1. Cleaner notation
2. Faster computation (NumPy is optimized for matrix operations)
3. Works for any number of features

Think of matrix multiplication as: **"for each sample, multiply each feature by its weight and sum them up"**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
np.random.seed(42)

## Step 1: Generate Sample Data

Let's create data with 3 features:
- Feature 1: House size (sq ft)
- Feature 2: Number of bedrooms
- Feature 3: House age (years)

$$\text{Price} = 50 \times \text{size} + 20 \times \text{bedrooms} - 5 \times \text{age} + 100 + \text{noise}$$

In [ ]:
# Generate sample data
n_samples = 200
n_features = 3

# Features: [size, bedrooms, age]
X = np.random.randn(n_samples, n_features)
X[:, 0] = X[:, 0] * 20 + 50  # Size: mean=50, std=20
X[:, 1] = np.abs(X[:, 1] * 1 + 3).astype(int)  # Bedrooms: around 3
X[:, 2] = np.abs(X[:, 2] * 10 + 15)  # Age: around 15 years

# True weights and bias
true_w = np.array([50, 20, -5])  # [size_weight, bedroom_weight, age_weight]
true_b = 100

# Generate target with noise
noise = np.random.normal(0, 200, n_samples)
y = X @ true_w + true_b + noise  # @ is matrix multiplication

print("Data Shape:")
print(f"X: {X.shape} (samples × features)")
print(f"y: {y.shape} (samples)")
print(f"\nTrue weights: {true_w}")
print(f"True bias: {true_b}")
print(f"\nFeature Statistics:")
print(f"Size (sq ft):  mean={X[:, 0].mean():.2f}, std={X[:, 0].std():.2f}")
print(f"Bedrooms:      mean={X[:, 1].mean():.2f}, std={X[:, 1].std():.2f}")
print(f"Age (years):   mean={X[:, 2].mean():.2f}, std={X[:, 2].std():.2f}")
print(f"Price:         mean={y.mean():.2f}, std={y.std():.2f}")

## Step 2: Visualize the Data

With multiple features, we can't plot everything in 2D anymore. Let's look at pairwise relationships.

In [ ]:
# Create a figure with subplots
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
feature_names = ['Size (sq ft)', 'Bedrooms', 'Age (years)']

for i in range(3):
    axes[i].scatter(X[:, i], y, alpha=0.6, s=50, edgecolors='black', linewidth=0.5)
    axes[i].set_xlabel(feature_names[i], fontsize=12)
    axes[i].set_ylabel('Price ($1000s)', fontsize=12)
    axes[i].set_title(f'{feature_names[i]} vs Price', fontsize=14, fontweight='bold')
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Step 3: Understanding the Math

### Prediction Formula

For a single sample:
- $\hat{y} = w_1x_1 + w_2x_2 + w_3x_3 + b$

For all samples at once (vectorized):
- $\hat{y} = Xw + b$

Where:
- $X$ shape: $(n_{samples}, n_{features})$
- $w$ shape: $(n_{features},)$
- Result shape: $(n_{samples},)$

### Mean Squared Error (MSE)

$$L = \frac{1}{n} \sum (y - \hat{y})^2 = \frac{1}{n} ||y - Xw - b||^2$$

### Computing Gradients

We need partial derivatives for **each weight** and the **bias**.

#### For weights $\left(\frac{\partial L}{\partial w}\right)$:

Let's derive this step by step:

1. $$L = \frac{1}{n} \sum_i \left(y_i - \left(\sum_j w_j x_{ij} + b\right)\right)^2$$

2. For a specific weight $w_k$:
   $$\frac{\partial L}{\partial w_k} = \frac{1}{n} \sum_i \left(2(y_i - \hat{y}_i) \times \frac{\partial \hat{y}_i}{\partial w_k}\right)$$
   
3. Since $\hat{y}_i = \sum_j w_j x_{ij} + b$:
   $$\frac{\partial \hat{y}_i}{\partial w_k} = x_{ik} \text{ (the k-th feature of sample i)}$$

4. Therefore:
   $$\boxed{\frac{\partial L}{\partial w_k} = \frac{-2}{n} \sum_i (y_i - \hat{y}_i) \times x_{ik}}$$

In matrix form:
$$\boxed{\frac{\partial L}{\partial w} = \frac{-2}{n} X^T(y - \hat{y})}$$

Where $X^T$ is the transpose of $X$ (swap rows and columns).

#### For bias $\left(\frac{\partial L}{\partial b}\right)$:

Same as before:
$$\boxed{\frac{\partial L}{\partial b} = \frac{-2}{n} \sum (y - \hat{y})}$$

### What does this mean?

- $\frac{\partial L}{\partial w_k}$: For each weight, we look at how that feature correlates with the error
  - If errors are positive when feature $x_k$ is large, we should decrease $w_k$
  - If errors are negative when feature $x_k$ is large, we should increase $w_k$

- $X^T(y - \hat{y})$: This efficiently computes all weight gradients at once!
  - Each element is the sum of (error $\times$ feature) for that feature
  - Matrix multiplication does this automatically

## Step 4: Implement Gradient Descent

In [ ]:
class LinearRegressionMultiVar:
    """Linear Regression with Multiple Variables using Gradient Descent"""
    
    def __init__(self, learning_rate=0.01, n_iterations=1000):
        self.lr = learning_rate
        self.n_iterations = n_iterations
        self.w = None  # weights
        self.b = None  # bias
        self.cost_history = []
        
    def compute_cost(self, X, y):
        """Calculate Mean Squared Error"""
        n = len(y)
        predictions = X @ self.w + self.b  # Matrix multiplication
        mse = (1/n) * np.sum((y - predictions) ** 2)
        return mse
    
    def compute_gradients(self, X, y):
        """Calculate partial derivatives for all weights and bias"""
        n = len(y)
        predictions = X @ self.w + self.b
        errors = y - predictions
        
        # Gradient for weights (vectorized!)
        # Shape: (n_features,)
        dw = (-2/n) * (X.T @ errors)
        
        # Gradient for bias
        db = (-2/n) * np.sum(errors)
        
        return dw, db
    
    def fit(self, X, y, verbose=True):
        """Train the model using gradient descent"""
        n_samples, n_features = X.shape
        
        # Initialize parameters
        self.w = np.random.randn(n_features) * 0.01
        self.b = 0.0
        
        print(f"Initial weights: {self.w}")
        print(f"Initial bias: {self.b:.4f}\n")
        
        # Gradient descent
        for i in range(self.n_iterations):
            # Calculate gradients
            dw, db = self.compute_gradients(X, y)
            
            # Update parameters
            self.w = self.w - self.lr * dw
            self.b = self.b - self.lr * db
            
            # Track cost
            cost = self.compute_cost(X, y)
            self.cost_history.append(cost)
            
            # Print progress
            if verbose and (i % 100 == 0 or i == self.n_iterations - 1):
                print(f"Iteration {i:4d} | Cost: {cost:12.2f} | weights: {self.w} | bias: {self.b:8.4f}")
        
        print(f"\nFinal weights: {self.w}")
        print(f"True weights:  {true_w}")
        print(f"\nFinal bias: {self.b:.4f}")
        print(f"True bias:  {true_b:.4f}")
        
    def predict(self, X):
        """Make predictions"""
        return X @ self.w + self.b

## Step 5: Feature Normalization (Important!)

### Why normalize?

Our features have very different scales:
- Size: 10-100
- Bedrooms: 1-5
- Age: 0-30

This causes problems:
1. Gradients for different features have different magnitudes
2. One learning rate doesn't work well for all features
3. Convergence is slower

### Solution: Standardization

Transform each feature to have:
- Mean = 0
- Standard deviation = 1

**Formula:** $$x_{\text{normalized}} = \frac{x - \mu}{\sigma}$$

where $\mu$ is the mean and $\sigma$ is the standard deviation.

This puts all features on the same scale!

In [ ]:
# Normalize features
X_mean = X.mean(axis=0)
X_std = X.std(axis=0)
X_normalized = (X - X_mean) / X_std

print("Before normalization:")
print(f"Mean: {X.mean(axis=0)}")
print(f"Std:  {X.std(axis=0)}")
print(f"\nAfter normalization:")
print(f"Mean: {X_normalized.mean(axis=0)}")
print(f"Std:  {X_normalized.std(axis=0)}")

## Step 6: Train the Model

In [ ]:
# Train with normalized features
model = LinearRegressionMultiVar(learning_rate=0.01, n_iterations=1000)
model.fit(X_normalized, y)

## Step 7: Visualize Results

In [ ]:
# Make predictions
y_pred = model.predict(X_normalized)

# Create visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Cost history
axes[0, 0].plot(model.cost_history, linewidth=2, color='#2E86AB')
axes[0, 0].set_xlabel('Iteration', fontsize=12)
axes[0, 0].set_ylabel('Cost (MSE)', fontsize=12)
axes[0, 0].set_title('Learning Curve', fontsize=14, fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Predictions vs Actual
axes[0, 1].scatter(y, y_pred, alpha=0.6, s=50, edgecolors='black', linewidth=0.5)
axes[0, 1].plot([y.min(), y.max()], [y.min(), y.max()], 'r--', linewidth=2, label='Perfect Prediction')
axes[0, 1].set_xlabel('Actual Price', fontsize=12)
axes[0, 1].set_ylabel('Predicted Price', fontsize=12)
axes[0, 1].set_title('Predictions vs Actual', fontsize=14, fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Residuals
residuals = y - y_pred
axes[1, 0].scatter(y_pred, residuals, alpha=0.6, s=50, edgecolors='black', linewidth=0.5)
axes[1, 0].axhline(y=0, color='r', linestyle='--', linewidth=2)
axes[1, 0].set_xlabel('Predicted Price', fontsize=12)
axes[1, 0].set_ylabel('Residuals (Actual - Predicted)', fontsize=12)
axes[1, 0].set_title('Residual Plot', fontsize=14, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Weight comparison
x_pos = np.arange(len(model.w))
width = 0.35
axes[1, 1].bar(x_pos - width/2, model.w, width, label='Learned Weights', alpha=0.8)
# Note: true_w needs to be adjusted for normalized data
true_w_normalized = true_w * X_std  # Adjust for normalization
axes[1, 1].bar(x_pos + width/2, true_w_normalized, width, label='True Weights (adjusted)', alpha=0.8)
axes[1, 1].set_xlabel('Feature Index', fontsize=12)
axes[1, 1].set_ylabel('Weight Value', fontsize=12)
axes[1, 1].set_title('Learned vs True Weights', fontsize=14, fontweight='bold')
axes[1, 1].set_xticks(x_pos)
axes[1, 1].set_xticklabels(['Size', 'Bedrooms', 'Age'])
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# Print metrics
mse = np.mean((y - y_pred) ** 2)
rmse = np.sqrt(mse)
r2 = 1 - (np.sum((y - y_pred) ** 2) / np.sum((y - y.mean()) ** 2))

print(f"\nPerformance Metrics:")
print(f"MSE:  {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²:   {r2:.4f}")

## Step 8: Understanding Weight Magnitudes

Let's interpret what our learned weights mean!

In [ ]:
# Convert normalized weights back to original scale
w_original_scale = model.w / X_std
b_original_scale = model.b - np.sum(w_original_scale * X_mean)

print("Weights in Original Scale:")
print(f"Size weight:     {w_original_scale[0]:8.4f} (true: {true_w[0]})")
print(f"Bedroom weight:  {w_original_scale[1]:8.4f} (true: {true_w[1]})")
print(f"Age weight:      {w_original_scale[2]:8.4f} (true: {true_w[2]})")
print(f"Bias:            {b_original_scale:8.4f} (true: {true_b})")

print("\nInterpretation:")
print(f"- Each additional sq ft increases price by ${w_original_scale[0]:.2f}k")
print(f"- Each additional bedroom increases price by ${w_original_scale[1]:.2f}k")
print(f"- Each additional year of age changes price by ${w_original_scale[2]:.2f}k")
print(f"- Base price (intercept) is ${b_original_scale:.2f}k")

## Step 9: Visualize 3D Decision Boundary (Using First 2 Features)

Let's visualize the relationship between 2 features and the target in 3D!

In [ ]:
# Create 3D plot using first 2 features
fig = plt.figure(figsize=(14, 6))

# Plot 1: Size vs Price (with bedroom color coding)
ax1 = fig.add_subplot(121, projection='3d')
scatter = ax1.scatter(X[:, 0], X[:, 1], y, c=y, cmap='viridis', 
                      alpha=0.6, s=50, edgecolors='black', linewidth=0.5)
ax1.set_xlabel('Size (sq ft)', fontsize=10)
ax1.set_ylabel('Bedrooms', fontsize=10)
ax1.set_zlabel('Price ($1000s)', fontsize=10)
ax1.set_title('3D Visualization: Size × Bedrooms → Price', fontsize=12, fontweight='bold')
plt.colorbar(scatter, ax=ax1, label='Price', shrink=0.5)

# Plot 2: Create a mesh for the regression plane
ax2 = fig.add_subplot(122, projection='3d')

# Create mesh
x1_range = np.linspace(X[:, 0].min(), X[:, 0].max(), 20)
x2_range = np.linspace(X[:, 1].min(), X[:, 1].max(), 20)
x1_mesh, x2_mesh = np.meshgrid(x1_range, x2_range)

# Use mean age for the 3rd feature
x3_mean = X[:, 2].mean()
X_mesh = np.column_stack([
    x1_mesh.ravel(),
    x2_mesh.ravel(),
    np.full(x1_mesh.size, x3_mean)
])
X_mesh_normalized = (X_mesh - X_mean) / X_std
y_mesh = model.predict(X_mesh_normalized).reshape(x1_mesh.shape)

# Plot surface
ax2.plot_surface(x1_mesh, x2_mesh, y_mesh, alpha=0.3, cmap='viridis')
ax2.scatter(X[:, 0], X[:, 1], y, c='red', alpha=0.5, s=30)
ax2.set_xlabel('Size (sq ft)', fontsize=10)
ax2.set_ylabel('Bedrooms', fontsize=10)
ax2.set_zlabel('Price ($1000s)', fontsize=10)
ax2.set_title('Regression Plane (Age=mean)', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

## Key Takeaways

### What We Learned:

1. **Multi-variable linear regression** extends single-variable to handle multiple features
   - Instead of: $y = mx + b$
   - We have: $y = w_1x_1 + w_2x_2 + \ldots + w_nx_n + b$

2. **Matrix notation** makes the math cleaner and computation faster
   - $y = Xw + b$
   - $X$ is $(n_{samples}, n_{features})$
   - $w$ is $(n_{features},)$
   - One matrix multiplication instead of many scalar multiplications!

3. **Vectorized gradients** compute all weight updates at once
   - $$\boxed{\frac{\partial L}{\partial w} = \frac{-2}{n} X^T(y - \hat{y})}$$
   - $X^T \cdot \text{errors}$ computes weighted sum of errors for each feature
   - Much faster than computing each weight gradient separately

4. **Feature normalization** is crucial
   - Different feature scales → different gradient magnitudes
   - Standardization (mean=0, std=1) fixes this
   - Faster convergence, one learning rate works for all features

5. **Matrix transpose** $(X^T)$:
   - Swaps rows and columns
   - Shape $(n_{samples}, n_{features}) \rightarrow (n_{features}, n_{samples})$
   - $X^T \cdot \text{errors}$: for each feature, sum up (error $\times$ feature_value)

### The Gradient Formula Explained:

$$\boxed{\frac{\partial L}{\partial w} = \frac{-2}{n} X^T(y - \hat{y})}$$

Breaking it down:
- $(y - \hat{y})$: Vector of errors for all samples
- $X^T$: Transpose of feature matrix
- $X^T(y - \hat{y})$: For each feature, compute sum of (error $\times$ feature_value)
- $\frac{-2}{n}$: Scaling factor (from derivative of squared error)

Think of it as:
"For each weight, how much does that feature correlate with the errors?"
- High positive correlation → increase weight
- High negative correlation → decrease weight
- No correlation → don't change weight

### Why This Matters for Neural Networks:

This is the foundation for deep learning!
- Neural networks are many linear regressions stacked together
- Each layer: $z = Xw + b$
- Backpropagation uses the same chain rule ideas
- Matrix operations scale to millions of parameters